# Controller Testing

Currently broken, not sure why openrocket_data.csv can't be read after fixing the Controls import

In [6]:
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
import sys

# We are currently in .../Maurice2/workspace
# We need to add the directory that CONTAINS "Maurice2" to sys.path (two levels up)
repo_parent = Path.cwd().resolve().parents[1]  # go up from workspace -> Maurice2 -> parent
if str(repo_parent) not in sys.path:
    sys.path.insert(0, str(repo_parent))

# now this works:
from Maurice2.controls.control_algorithm import Controls

In [8]:
## Define gain matrix ##
Kmax_preburnout = 100 / 7e1
Kmin_preburnout = 17.5 / 7e1

K_max_postburnout = 85 / 6e1
K_min_postburnout = 17.5 / 6e1

Ks = np.array([Kmax_preburnout, Kmin_preburnout, K_max_postburnout, K_min_postburnout])  # Gain scheduling based on altitude

## Define initial conditions ##
t0 = 0.0
xhat0 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0]) # Initial state estimate
u0 = np.array([0])
sampling_rate = 20.0  # Hz
dt = 1.0 / sampling_rate

controller = Controls(Ks=Ks, dt=dt, x0=xhat0, u0=u0, t_launch_rail_clearance=0.308)
controller.deriveEOM(post_burnout=False)
controller.deriveEOM(post_burnout=True)
controller.buildL(lw=5.0, lqw=1.0, lqx=2.0, lqy=2.0, lqz=2.0)
controller.test_AB(t0, xhat0, u0)

t: 0.000, xhat: [0 0 0 0 0 0 1 0 0 0], u: [0]


FileNotFoundError: [Errno 2] No such file or directory: 'Maurice2/data/openrocket_data.csv'

In [ ]:
xhat = np.array(controller.states)
u = np.array(controller.inputs)
t = np.arange(0, xhat.shape[0]*controller.dt, controller.dt)

fig, axs = plt.subplots(4, 1, figsize=(10, 8), sharex=False)
fig.suptitle('Rocket State Estimator Results')
axs[0].plot(t, xhat[:, 0], label='w1')
axs[0].plot(t, xhat[:, 1], label='w2')
axs[0].plot(t, xhat[:, 2], label='w3')
axs[0].set_ylim([-1e-3, 1e-2])
v3_100_indices = np.where(xhat[:, 5] >= 100)[0]
v3_100_time = v3_100_indices[0] * controller.dt
axs[0].axvline(v3_100_time, color='m', linestyle='--', label='v3 = 100 m/s')
axs[0].axvline(controller.t_motor_burnout, color='r', linestyle='--', label='Motor Burnout')
axs[0].set_ylabel('Angular Velocities (rad/s)')
axs[0].legend(loc='upper right')
axs[0].grid()
axs[1].plot(t, xhat[:, 3], label='v1')
axs[1].plot(t, xhat[:, 4], label='v2')
axs[1].plot(t, xhat[:, 5], label='v3')
axs[1].axvline(v3_100_time, color='m', linestyle='--', label='v3 = 100 m/s')
axs[1].axvline(controller.t_motor_burnout, color='r', linestyle='--', label='Motor Burnout')
axs[1].set_ylabel('Linear Velocities (m/s)')
axs[1].legend(loc='upper right')
axs[1].grid()
axs[2].plot(t, u * 180/np.pi, label='delta1')
axs[2].plot(t, -8 * np.ones_like(u), 'b--', label='Aileron Limits')
axs[2].plot(t, 8 * np.ones_like(u), 'b--')
axs[2].axvline(v3_100_time, color='m', linestyle='--', label='v3 = 100 m/s')
axs[2].axvline(controller.t_motor_burnout, color='r', linestyle='--', label='Motor Burnout')
axs[2].set_ylabel('Aileron Deflection (°)')
axs[2].set_xlabel('Time (s)')
axs[2].legend(loc='upper right')
axs[2].grid()

# Convert quaternion to Euler angles for plotting
euler_angles = np.array([controller.quat_to_euler_xyz(q) for q in xhat[:, 6:10]])
axs[3].plot(t, np.rad2deg(euler_angles[:, 0]), label='Pitch (θ)')
axs[3].plot(t, np.rad2deg(euler_angles[:, 1]), label='Yaw (φ)')
axs[3].plot(t, np.rad2deg(euler_angles[:, 2]), label='Roll (ψ)')
axs[3].axvline(controller.t_motor_burnout, color='r', linestyle='--', label='Motor Burnout')
axs[3].set_ylabel('Euler Angles (°)')
axs[3].set_xlabel('Time (s)')
axs[3].legend(loc='upper right')
axs[3].grid()
plt.tight_layout()

# # Alternative: Plot quaternion components ##
# axs[3].plot(t, xhat[:, 6], label='qw')
# axs[3].plot(t, xhat[:, 7], label='qx')
# axs[3].plot(t, xhat[:, 8], label='qy')
# axs[3].plot(t, xhat[:, 9], label='qz')
# axs[3].axvline(controller.t_motor_burnout, color='r', linestyle='--', label='Motor Burnout')
# axs[3].set_ylabel('Quaternion Components')
# axs[3].set_xlabel('Time (s)')
# axs[3].legend()
# axs[3].grid()
# plt.tight_layout()